In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.model_selection import KFold, train_test_split
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error

data = pd.read_excel('data/data_io.xlsx', names = ['time (min)', 'm_xylene', 'NO', 'NO2', 'NOx', 'NOx/m_xy', 'M0', 'yield'])
data.drop(columns = ['NO', 'NO2', 'NOx/m_xy'], inplace = True)
#data['NOx'] = 1 / data['NOx']
scaler = StandardScaler()

In [ ]:
X = scaler.fit_transform(data[data.columns[:-2]])
y1 = data[data.columns[-2]].to_numpy()
y2 = data[data.columns[-1]].to_numpy()
X.shape

In [ ]:
from statsmodels.regression.linear_model import OLS
import statsmodels.api as sm
# yields
X = sm.add_constant(X)
model = OLS(y1, X).fit()
model.params

In [ ]:
model.summary()

In [ ]:
# M0
model = OLS(y2, X).fit()
model.params

In [ ]:
model.summary()

In [ ]:
def percent_error(actual, pred):
    return np.mean(np.absolute((actual - pred) / pred) * 100)

In [ ]:
k = 5
kf = KFold(n_splits=k, shuffle=True)

mse_scores = []
percent_errs = []

for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y1[train_index], y1[val_index]

    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)

    model = Ridge(alpha=0.1)

    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)

    mse = mean_squared_error(y_val, y_pred)
    percent_err = percent_error(y_val, y_pred)
    mse_scores.append(mse)
    percent_errs.append(percent_err)

avg_mse = np.mean(mse_scores)
avg_percent_err = np.mean(percent_errs)




In [ ]:
# 1 / NOx, more consistent, less error
avg_mse, avg_percent_err

In [ ]:
k = 5
kf = KFold(n_splits=k, shuffle=True)

mae_scores = []
percent_errs = []

for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y2[train_index], y2[val_index]

    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)

    model = Ridge(alpha=0.1)

    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)

    mae = mean_absolute_error(y_val, y_pred)
    percent_err = percent_error(y_val, y_pred)
    mae_scores.append(mae)
    percent_errs.append(percent_err)

avg_mae = np.mean(mae_scores)
avg_percent_err = np.mean(percent_errs)

In [ ]:
avg_mae, avg_percent_err

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y1, test_size = 0.2, random_state = 10)
model = Ridge(alpha = 0.1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred

In [ ]:
y_test

In [ ]:
percent_error(y_test, y_pred)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y2, test_size = 0.2, random_state = 20)
model = Ridge(alpha = 0.1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred

In [ ]:
y_test

In [ ]:
percent_error(y_test, y_pred)

# PCA

In [ ]:
data = pd.read_excel('data/data_io.xlsx', names = ['time (min)', 'm_xylene', 'NO', 'NO2', 'NOx', 'NOx/m_xy', 'M0', 'yield'])
#data = pd.read_excel('data/data_temp.xlsx', names = ['time (min)', 'm_xylene', 'NO', 'NO2', 'NOx', 'NOx/m_xy', 'wall loss factor', 'M0', 'yield'])
X = data[data.columns[:-2]]
y = data[data.columns[-2:]]
y1 = data[data.columns[-2]]
y2 = data[data.columns[-1]]

In [ ]:
n_components = 3
X_scaled = scaler.fit_transform(X)
pca = PCA(n_components = n_components)
X_pca = pca.fit_transform(X_scaled)

ratio = pca.explained_variance_ratio_
f'Percent of Variance Captured by {n_components} Principal Components: {ratio.sum():.2%}'

In [ ]:
loadings = pca.components_
loadings = np.round(pd.DataFrame(loadings, columns = X.columns), 3)
plt.figure(figsize = (12, 6))
sns.set(font_scale = 1.2)
sns.set_style('whitegrid')
ax = sns.heatmap(loadings, annot = True, cmap = 'coolwarm', center = 0, linewidths = 0.5,
            cbar_kws = {'shrink': 0.8, 'aspect': 10, 'pad': 0.02},
            annot_kws = {'fontsize': 12})

ax.tick_params(axis = 'both', labelsize = 12)
ax.set_title('PCA Feature Influence', fontsize = 16, pad = 15)
ax.set_xlabel('Features', fontsize = 14, labelpad = 10)
ax.set_ylabel('Principal Components', fontsize = 14, labelpad = 10)

ax.yaxis.set_ticklabels(ax.yaxis.get_ticklabels(), rotation=0, ha='right')

plt.tight_layout()
plt.show()

In [ ]:
pca.explained_variance_ratio_

In [ ]:
fig, ax = plt.subplots()

scatter = ax.scatter(X_pca[:, 0], X_pca[:, 1], c=y1, cmap='viridis')

cbar = fig.colorbar(scatter)
cbar.set_label('M0')

ax.set_xlabel('PCA Component 1')
ax.set_ylabel('PCA Component 2')
ax.set_title('PCA Components with Heatmap for M0')

plt.show()

In [ ]:
fig, ax = plt.subplots()

scatter = ax.scatter(X_pca[:, 0], X_pca[:, 1], c=y2, cmap='viridis')

cbar = fig.colorbar(scatter)
cbar.set_label('yield')

ax.set_xlabel('PCA Component 1')
ax.set_ylabel('PCA Component 2')
ax.set_title('PCA Components with Heatmap for yield')

plt.show()

In [ ]:
model = OLS(y1, X_pca).fit()
model.summary()

In [ ]:
model = OLS(y2, X_pca).fit()
model.summary()

In [ ]:
x = pd.DataFrame(X_pca)
x.corr()

# T-SNE

In [ ]:
tsne = TSNE(n_components = 2, random_state = 10)

X_tsne = tsne.fit_transform(X)

model = OLS(y1, X_tsne).fit()
model.summary()

In [ ]:
# suggests non-linear relationship not captured by linear models
model = OLS(y2, X_tsne).fit()
model.summary()

In [ ]:
fig, ax = plt.subplots()

scatter = ax.scatter(X_tsne[:, 0], X_tsne[:, 1], c=y1, cmap='viridis')

# Add a colorbar
cbar = fig.colorbar(scatter)
cbar.set_label('M0')

ax.set_xlabel('t-SNE Component 1')
ax.set_ylabel('t-SNE Component 2')
ax.set_title('t-SNE Components with Heatmap for M0')

plt.show()


In [ ]:
fig, ax = plt.subplots()

scatter = ax.scatter(X_tsne[:, 0], X_tsne[:, 1], c=y2, cmap='viridis')

cbar = fig.colorbar(scatter)
cbar.set_label('yield')

ax.set_xlabel('t-SNE Component 1')
ax.set_ylabel('t-SNE Component 2')
ax.set_title('t-SNE Components with Heatmap for yield')

plt.show()

In [ ]:
outlier_indices = np.where(X_tsne[:, 0] > -0.25)
outliers = X.loc[outlier_indices]

In [ ]:
outliers

In [ ]:
outliers.mean()

In [ ]:
with pd.ExcelWriter('written_data/temp.xlsx') as file:
    outliers.mean().to_excel(file)

In [ ]:
mask = ~X.index.isin(list(outlier_indices[0]))
non_outliers = X.loc[mask]
non_outliers

In [ ]:
non_outliers.mean()

In [ ]:
with pd.ExcelWriter('written_data/temp.xlsx') as file:
    non_outliers.mean().to_excel(file)

In [ ]:
outlier_indices = np.where(X_pca[:, 0] > 0.4)
X.loc[outlier_indices[0]]

In [ ]:
X.loc[outlier_indices[0]].mean()